# 📚 Sincronização de Biblioteca de Ebooks com Google Sheets

Este notebook permite criar e manter atualizada uma planilha Google Sheets com todos os ebooks da sua biblioteca no Google Drive.

**Características:**
- ✅ Suporta bibliotecas grandes (50k+ arquivos)
- ✅ Varredura recursiva de todas as subpastas
- ✅ Cache inteligente para atualizações incrementais
- ✅ Retry automático em caso de erros de rede
- ✅ Logging detalhado do processo
- ✅ Formatação automática da planilha

**Formatos suportados:** PDF, EPUB, MOBI, AZW, AZW3, DJVU, FB2, TXT, RTF, DOC, DOCX, CBR, CBZ, LIT

---

## 🔧 Passo 1: Configuração Inicial

### 1.1 Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 1.2 Instalar Bibliotecas Necessárias

In [ ]:
!pip install -q google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
print("✓ Bibliotecas instaladas com sucesso!")

## ⚙️ Passo 2: Configuração

**IMPORTANTE:** Ajuste os valores abaixo conforme sua configuração

In [ ]:
# ========== CONFIGURAÇÕES - AJUSTE AQUI ==========

# Caminho para o arquivo de credenciais do Google Drive
CREDENTIALS_PATH = "/content/drive/MyDrive/0_Credentials/acessodriveorlando-44351dfb71f4.json"

# ID da pasta raiz da biblioteca no Google Drive
LIBRARY_FOLDER_ID = "0B9gSg9OIekOlajlGdWcxOWt0MlU"

# ID da planilha existente (deixe None para criar nova planilha)
# Se você já tem uma planilha e quer atualizar, cole o ID aqui
# Exemplo: "1a2b3c4d5e6f7g8h9i0j"
SPREADSHEET_ID = None  # None = criar nova planilha

# Nome da aba na planilha
SHEET_NAME = "Biblioteca de Ebooks"

# Arquivo de cache (para sincronizações incrementais futuras)
CACHE_FILE = "/content/drive/MyDrive/library_cache.pkl"

# ==================================================

### 2.1 Verificar Credenciais

In [ ]:
import os
import json

# Verificar se arquivo de credenciais existe
if os.path.exists(CREDENTIALS_PATH):
    print(f"✓ Arquivo de credenciais encontrado: {CREDENTIALS_PATH}")
    
    # Mostrar informações básicas
    with open(CREDENTIALS_PATH, 'r') as f:
        creds = json.load(f)
        print(f"  - Tipo: {creds.get('type', 'N/A')}")
        print(f"  - Projeto: {creds.get('project_id', 'N/A')}")
        print(f"  - Email: {creds.get('client_email', 'N/A')}")
else:
    print(f"✗ ERRO: Arquivo de credenciais não encontrado: {CREDENTIALS_PATH}")
    print("  Verifique o caminho e tente novamente.")

## 📝 Passo 3: Código do Script

Execute a célula abaixo para carregar o código do sincronizador

In [ ]:
import os
import json
import pickle
import time
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Optional, Set
import logging
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Configuração de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class EbookLibrarySync:
    """Classe para sincronizar biblioteca de ebooks com Google Sheets"""

    DRIVE_API_BATCH_SIZE = 1000
    SHEETS_BATCH_SIZE = 10000
    MAX_RETRIES = 5
    RETRY_DELAY = 2

    EBOOK_EXTENSIONS = {
        '.pdf', '.epub', '.mobi', '.azw', '.azw3', '.djvu', '.fb2',
        '.txt', '.rtf', '.doc', '.docx', '.cbr', '.cbz', '.lit'
    }

    def __init__(self, credentials_path: str, library_folder_id: str,
                 cache_file: str = 'library_cache.pkl'):
        self.credentials_path = credentials_path
        self.library_folder_id = library_folder_id
        self.cache_file = cache_file
        self.drive_service = None
        self.sheets_service = None
        self.cache = self._load_cache()
        logger.info("Inicializando EbookLibrarySync...")
        self._authenticate()

    def _authenticate(self):
        try:
            logger.info(f"Autenticando com credenciais: {self.credentials_path}")
            SCOPES = [
                'https://www.googleapis.com/auth/drive.readonly',
                'https://www.googleapis.com/auth/spreadsheets'
            ]
            credentials = service_account.Credentials.from_service_account_file(
                self.credentials_path, scopes=SCOPES
            )
            self.drive_service = build('drive', 'v3', credentials=credentials)
            self.sheets_service = build('sheets', 'v4', credentials=credentials)
            logger.info("✓ Autenticação bem-sucedida!")
        except Exception as e:
            logger.error(f"✗ Erro na autenticação: {e}")
            raise

    def _load_cache(self) -> Dict:
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'rb') as f:
                    cache = pickle.load(f)
                logger.info(f"Cache carregado: {len(cache.get('files', {}))} arquivos")
                return cache
            except Exception as e:
                logger.warning(f"Erro ao carregar cache: {e}")
        return {'files': {}, 'last_sync': None}

    def _save_cache(self):
        try:
            self.cache['last_sync'] = datetime.now().isoformat()
            with open(self.cache_file, 'wb') as f:
                pickle.dump(self.cache, f)
            logger.info(f"Cache salvo: {len(self.cache['files'])} arquivos")
        except Exception as e:
            logger.error(f"Erro ao salvar cache: {e}")

    def _retry_request(self, func, *args, **kwargs):
        for attempt in range(self.MAX_RETRIES):
            try:
                return func(*args, **kwargs)
            except HttpError as e:
                if e.resp.status in [403, 429, 500, 503]:
                    wait_time = self.RETRY_DELAY * (2 ** attempt)
                    logger.warning(f"Erro {e.resp.status}, tentando novamente em {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    raise
            except Exception as e:
                if attempt < self.MAX_RETRIES - 1:
                    time.sleep(self.RETRY_DELAY)
                else:
                    raise
        raise Exception(f"Falha após {self.MAX_RETRIES} tentativas")

    def _list_files_in_folder(self, folder_id: str, page_token: Optional[str] = None) -> Dict:
        query = f"'{folder_id}' in parents and trashed=false"
        fields = "nextPageToken, files(id, name, mimeType, size, createdTime, modifiedTime, webViewLink, parents)"
        try:
            results = self._retry_request(
                self.drive_service.files().list,
                q=query,
                pageSize=self.DRIVE_API_BATCH_SIZE,
                fields=fields,
                pageToken=page_token
            ).execute()
            return results
        except Exception as e:
            logger.error(f"Erro ao listar arquivos da pasta {folder_id}: {e}")
            return {'files': [], 'nextPageToken': None}

    def _get_folder_path(self, file_id: str, file_name: str, cache_paths: Dict[str, str]) -> str:
        if file_id in cache_paths:
            return cache_paths[file_id]
        try:
            file_info = self._retry_request(
                self.drive_service.files().get,
                fileId=file_id,
                fields='parents'
            ).execute()
            parents = file_info.get('parents', [])
            if not parents or parents[0] == self.library_folder_id:
                cache_paths[file_id] = f"/{file_name}"
                return cache_paths[file_id]
            parent_id = parents[0]
            parent_info = self._retry_request(
                self.drive_service.files().get,
                fileId=parent_id,
                fields='name, parents'
            ).execute()
            parent_name = parent_info.get('name', 'Unknown')
            parent_path = self._get_folder_path(parent_id, parent_name, cache_paths)
            full_path = f"{parent_path}/{file_name}"
            cache_paths[file_id] = full_path
            return full_path
        except Exception as e:
            logger.error(f"Erro ao obter caminho do arquivo {file_id}: {e}")
            return f"/ERROR/{file_name}"

    def scan_library(self, progress_callback=None) -> List[Dict]:
        logger.info("Iniciando varredura da biblioteca...")
        all_files = []
        folders_to_process = [self.library_folder_id]
        processed_folders = set()
        cache_paths = {}
        total_files = 0
        total_folders = 0

        while folders_to_process:
            current_folder_id = folders_to_process.pop(0)
            if current_folder_id in processed_folders:
                continue
            processed_folders.add(current_folder_id)
            total_folders += 1
            logger.info(f"Processando pasta {total_folders}... (arquivos: {total_files})")

            page_token = None
            while True:
                results = self._list_files_in_folder(current_folder_id, page_token)
                files = results.get('files', [])
                
                for file_info in files:
                    file_id = file_info['id']
                    file_name = file_info['name']
                    mime_type = file_info['mimeType']
                    
                    if mime_type == 'application/vnd.google-apps.folder':
                        folders_to_process.append(file_id)
                        continue
                    
                    extension = Path(file_name).suffix.lower()
                    if extension not in self.EBOOK_EXTENSIONS:
                        continue
                    
                    file_path = self._get_folder_path(file_id, file_name, cache_paths)
                    
                    file_data = {
                        'id': file_id,
                        'nome': file_name,
                        'caminho': file_path,
                        'extensao': extension,
                        'tamanho': int(file_info.get('size', 0)),
                        'tamanho_mb': round(int(file_info.get('size', 0)) / (1024 * 1024), 2),
                        'data_criacao': file_info.get('createdTime', ''),
                        'data_modificacao': file_info.get('modifiedTime', ''),
                        'link': file_info.get('webViewLink', ''),
                        'mime_type': mime_type
                    }
                    
                    all_files.append(file_data)
                    total_files += 1
                    
                    if progress_callback and total_files % 100 == 0:
                        progress_callback(total_files, total_folders)
                
                page_token = results.get('nextPageToken')
                if not page_token:
                    break
                time.sleep(0.1)

        logger.info(f"✓ Varredura concluída: {total_files} ebooks em {total_folders} pastas")
        return all_files

    def create_or_update_spreadsheet(self, files_data: List[Dict],
                                     spreadsheet_id: Optional[str] = None,
                                     sheet_name: str = "Biblioteca de Ebooks") -> str:
        logger.info(f"{'Atualizando' if spreadsheet_id else 'Criando'} planilha...")
        
        if not spreadsheet_id:
            spreadsheet_id = self._create_spreadsheet(sheet_name)
        
        headers = [
            'ID', 'Nome', 'Caminho', 'Extensão', 'Tamanho (bytes)',
            'Tamanho (MB)', 'Data Criação', 'Data Modificação', 'Link'
        ]
        
        rows = [headers]
        for file_data in files_data:
            row = [
                file_data['id'],
                file_data['nome'],
                file_data['caminho'],
                file_data['extensao'],
                file_data['tamanho'],
                file_data['tamanho_mb'],
                file_data['data_criacao'],
                file_data['data_modificacao'],
                file_data['link']
            ]
            rows.append(row)
        
        self._update_sheet_in_batches(spreadsheet_id, sheet_name, rows)
        self._format_spreadsheet(spreadsheet_id, sheet_name, len(rows))
        
        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}"
        logger.info(f"✓ Planilha atualizada: {spreadsheet_url}")
        return spreadsheet_id

    def _create_spreadsheet(self, title: str) -> str:
        try:
            spreadsheet = {
                'properties': {
                    'title': f'{title} - {datetime.now().strftime("%Y-%m-%d %H:%M")}'
                }
            }
            spreadsheet = self._retry_request(
                self.sheets_service.spreadsheets().create,
                body=spreadsheet,
                fields='spreadsheetId'
            ).execute()
            return spreadsheet.get('spreadsheetId')
        except Exception as e:
            logger.error(f"✗ Erro ao criar planilha: {e}")
            raise

    def _update_sheet_in_batches(self, spreadsheet_id: str, sheet_name: str, rows: List[List]):
        total_rows = len(rows)
        logger.info(f"Atualizando planilha com {total_rows} linhas...")
        try:
            self._retry_request(
                self.sheets_service.spreadsheets().values().clear,
                spreadsheetId=spreadsheet_id,
                range=sheet_name,
                body={}
            ).execute()
            
            for i in range(0, total_rows, self.SHEETS_BATCH_SIZE):
                batch = rows[i:i + self.SHEETS_BATCH_SIZE]
                end_row = min(i + self.SHEETS_BATCH_SIZE, total_rows)
                logger.info(f"Atualizando linhas {i+1} a {end_row}...")
                body = {'values': batch}
                self._retry_request(
                    self.sheets_service.spreadsheets().values().update,
                    spreadsheetId=spreadsheet_id,
                    range=f"{sheet_name}!A{i+1}",
                    valueInputOption='RAW',
                    body=body
                ).execute()
                time.sleep(0.5)
            logger.info(f"✓ {total_rows} linhas atualizadas!")
        except Exception as e:
            logger.error(f"✗ Erro ao atualizar planilha: {e}")
            raise

    def _format_spreadsheet(self, spreadsheet_id: str, sheet_name: str, num_rows: int):
        try:
            spreadsheet = self._retry_request(
                self.sheets_service.spreadsheets().get,
                spreadsheetId=spreadsheet_id
            ).execute()
            
            sheet_id = None
            for sheet in spreadsheet['sheets']:
                if sheet['properties']['title'] == sheet_name:
                    sheet_id = sheet['properties']['sheetId']
                    break
            
            if not sheet_id:
                return
            
            requests = [
                {
                    'updateSheetProperties': {
                        'properties': {
                            'sheetId': sheet_id,
                            'gridProperties': {'frozenRowCount': 1}
                        },
                        'fields': 'gridProperties.frozenRowCount'
                    }
                },
                {
                    'repeatCell': {
                        'range': {
                            'sheetId': sheet_id,
                            'startRowIndex': 0,
                            'endRowIndex': 1
                        },
                        'cell': {
                            'userEnteredFormat': {
                                'textFormat': {'bold': True},
                                'backgroundColor': {'red': 0.9, 'green': 0.9, 'blue': 0.9}
                            }
                        },
                        'fields': 'userEnteredFormat(textFormat,backgroundColor)'
                    }
                },
                {
                    'autoResizeDimensions': {
                        'dimensions': {
                            'sheetId': sheet_id,
                            'dimension': 'COLUMNS',
                            'startIndex': 0,
                            'endIndex': 9
                        }
                    }
                }
            ]
            
            self._retry_request(
                self.sheets_service.spreadsheets().batchUpdate,
                spreadsheetId=spreadsheet_id,
                body={'requests': requests}
            ).execute()
            logger.info("✓ Planilha formatada!")
        except Exception as e:
            logger.error(f"⚠ Erro ao formatar planilha: {e}")

    def sync(self, spreadsheet_id: Optional[str] = None) -> str:
        start_time = time.time()
        logger.info("=" * 60)
        logger.info("INICIANDO SINCRONIZAÇÃO DA BIBLIOTECA")
        logger.info("=" * 60)
        
        def progress(files, folders):
            logger.info(f"Progresso: {files} ebooks em {folders} pastas")
        
        files_data = self.scan_library(progress_callback=progress)
        
        if not files_data:
            logger.warning("Nenhum ebook encontrado!")
            return None
        
        self.cache['files'] = {f['id']: f for f in files_data}
        self._save_cache()
        
        spreadsheet_id = self.create_or_update_spreadsheet(files_data, spreadsheet_id)
        
        elapsed_time = time.time() - start_time
        total_size_gb = sum(f['tamanho'] for f in files_data) / (1024 ** 3)
        
        logger.info("=" * 60)
        logger.info("SINCRONIZAÇÃO CONCLUÍDA!")
        logger.info(f"Total de ebooks: {len(files_data)}")
        logger.info(f"Tamanho total: {total_size_gb:.2f} GB")
        logger.info(f"Tempo: {elapsed_time:.2f} segundos")
        logger.info(f"URL: https://docs.google.com/spreadsheets/d/{spreadsheet_id}")
        logger.info("=" * 60)
        
        return spreadsheet_id

print("✓ Classe EbookLibrarySync carregada com sucesso!")

## 🚀 Passo 4: Executar Sincronização

Execute a célula abaixo para iniciar a sincronização

In [ ]:
# Criar instância do sincronizador
sync = EbookLibrarySync(
    credentials_path=CREDENTIALS_PATH,
    library_folder_id=LIBRARY_FOLDER_ID,
    cache_file=CACHE_FILE
)

# Executar sincronização
spreadsheet_id = sync.sync(spreadsheet_id=SPREADSHEET_ID)

# Exibir resultado
if spreadsheet_id:
    spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}"
    print("\n" + "="*60)
    print("🎉 SINCRONIZAÇÃO CONCLUÍDA COM SUCESSO!")
    print("="*60)
    print(f"\n📊 Sua planilha está disponível em:")
    print(f"\n{spreadsheet_url}")
    print(f"\n💡 Dica: Salve o ID da planilha para futuras atualizações:")
    print(f"   SPREADSHEET_ID = \"{spreadsheet_id}\"")
    print("\n" + "="*60)
else:
    print("\n⚠ Nenhum ebook encontrado na biblioteca.")

## 📊 Passo 5: Estatísticas (Opcional)

Execute para ver estatísticas detalhadas da sua biblioteca

In [ ]:
import pandas as pd
from collections import Counter

# Carregar dados do cache
if sync.cache['files']:
    files_data = list(sync.cache['files'].values())
    
    # Estatísticas gerais
    total_files = len(files_data)
    total_size_gb = sum(f['tamanho'] for f in files_data) / (1024 ** 3)
    
    print("📊 ESTATÍSTICAS DA BIBLIOTECA")
    print("="*60)
    print(f"Total de ebooks: {total_files:,}")
    print(f"Tamanho total: {total_size_gb:.2f} GB")
    print(f"Tamanho médio: {(total_size_gb * 1024 / total_files):.2f} MB")
    
    # Por extensão
    extensions = Counter(f['extensao'] for f in files_data)
    print("\n📚 Por formato:")
    for ext, count in extensions.most_common():
        percentage = (count / total_files) * 100
        print(f"  {ext}: {count:,} ({percentage:.1f}%)")
    
    # Top 10 maiores arquivos
    print("\n📦 Top 10 maiores arquivos:")
    sorted_files = sorted(files_data, key=lambda x: x['tamanho'], reverse=True)[:10]
    for i, f in enumerate(sorted_files, 1):
        print(f"  {i}. {f['nome']} ({f['tamanho_mb']} MB)")
    
    print("\n" + "="*60)
else:
    print("⚠ Nenhum dado no cache. Execute a sincronização primeiro.")

## 🔧 Operações Avançadas

### Buscar arquivos específicos

In [ ]:
# Buscar por palavra-chave no nome do arquivo
search_term = "python"  # Altere aqui

if sync.cache['files']:
    results = [
        f for f in sync.cache['files'].values()
        if search_term.lower() in f['nome'].lower()
    ]
    
    print(f"🔍 Resultados para '{search_term}': {len(results)} arquivo(s)\n")
    
    for f in results[:20]:  # Mostrar primeiros 20
        print(f"📖 {f['nome']}")
        print(f"   📁 {f['caminho']}")
        print(f"   📊 {f['tamanho_mb']} MB")
        print(f"   🔗 {f['link']}")
        print()
else:
    print("⚠ Execute a sincronização primeiro.")

## 💡 Dicas e Informações

### Como atualizar a planilha existente:
1. Copie o ID da planilha da URL (parte após `/d/`)
2. Defina `SPREADSHEET_ID = "seu-id-aqui"` na seção de configurações
3. Execute novamente a sincronização

### Limites do Google Sheets:
- Máximo de 10 milhões de células
- Com 9 colunas, você pode ter até ~1.1 milhão de linhas
- Sua biblioteca com 200k arquivos está bem dentro do limite

### Cache:
- O cache é salvo no seu Drive para acelerar sincronizações futuras
- Ele ajuda a identificar arquivos novos, modificados ou removidos
- Para forçar varredura completa, delete o arquivo de cache

### Performance:
- Para 50k arquivos: ~30-60 minutos
- Para 100k arquivos: ~60-120 minutos
- O tempo varia conforme a estrutura de pastas e conexão

### Compartilhamento da planilha:
Para compartilhar a planilha, você precisa dar permissão ao email da service account:
- Email: disponível nas credenciais JSON (`client_email`)
- Abra a planilha > Compartilhar > Adicione o email
- Ou torne a planilha pública (qualquer pessoa com o link)

---

**Criado com ❤️ por Claude**